In [4]:
! chmod 600 kaggle.json
! KAGGLE_CONFIG_DIR=/content/  kaggle datasets download -d hchen13/marvel-heroes

 99% 1.13G/1.13G [00:07<00:00, 170MB/s]
100% 1.13G/1.13G [00:07<00:00, 154MB/s]


**Mempersiapakan Direktori**

In [5]:
import zipfile 
import os

In [6]:
dataZip = zipfile.ZipFile('/content/marvel-heroes.zip')
dataZip.extractall('/content/')

In [7]:
base_dir = '/content/marvel/train'
hero_class = os.listdir(base_dir)
hero_class

['loki',
 'thanos',
 'ironman',
 'doctor strange',
 'black widow',
 'spider-man',
 'captain america',
 'hulk']

**Image Preprocessing**

In [8]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 30,
    horizontal_flip = True,
    vertical_flip = True,
    shear_range = 0.2,
    fill_mode = 'nearest',
    width_shift_range = .2,
    height_shift_range = .2,
    validation_split = 0.2
)

In [10]:
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size = (100,150),
    classes = hero_class,
    class_mode = 'categorical',
    subset = 'training',
    batch_size = 4,
    shuffle = False
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size = (100,150),
    classes = hero_class,
    class_mode = 'categorical',
    subset = 'validation',
    batch_size = 3,
    shuffle = True
)

Found 2070 images belonging to 8 classes.
Found 514 images belonging to 8 classes.


In [11]:
print(train_generator[0])

(array([[[[0.59793997, 0.5038223 , 0.40186158],
         [0.5983007 , 0.50418305, 0.40222225],
         [0.59866136, 0.5045437 , 0.4025829 ],
         ...,
         [0.11473218, 0.14017737, 0.20006417],
         [0.10316685, 0.15217558, 0.22953936],
         [0.09177963, 0.15575837, 0.22323413]],

        [[0.6019958 , 0.51365554, 0.40591735],
         [0.60187554, 0.513896  , 0.40579712],
         [0.6017553 , 0.5141365 , 0.4056769 ],
         ...,
         [0.11680613, 0.13959251, 0.21639828],
         [0.11247475, 0.15749273, 0.24754855],
         [0.0953813 , 0.15481675, 0.2439413 ]],

        [[0.6115435 , 0.52534264, 0.40776938],
         [0.6122648 , 0.5258236 , 0.40800983],
         [0.61298615, 0.5263045 , 0.4082503 ],
         ...,
         [0.11373277, 0.1416764 , 0.20767562],
         [0.11986019, 0.1556143 , 0.243505  ],
         [0.11380866, 0.15688407, 0.25294325]],

        ...,

        [[0.07843138, 0.16078432, 0.23529413],
         [0.07843138, 0.16078432, 0.23529413

**Modelling**

In [12]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(3,3), activation='relu', input_shape = (100, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(200, activation = 'relu'),
    tf.keras.layers.Dropout(0.3, seed=112),
    tf.keras.layers.Dense(500, activation = 'relu'),
    tf.keras.layers.Dropout(0.5, seed=112),
    tf.keras.layers.Dense(8, activation = 'softmax')

])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 148, 32)       896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 34, 32)        18464     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 17, 32)       0

In [13]:
model.compile(
    loss= 'categorical_crossentropy',
    optimizer = 'Adam',
    metrics = ['accuracy']
)


In [14]:
result = model.fit(
    train_generator,
    steps_per_epoch = 20,
    epochs = 10,
    validation_steps = 5,
    validation_data = validation_generator,
    verbose = 1,
    batch_size=8,
)


Epoch 1/10
20/20 [==============================] - 11s 119ms/step - loss: 2.1593 - accuracy: 0.0625 - val_loss: 2.0788 - val_accuracy: 0.2000
Epoch 2/10
20/20 [==============================] - 3s 152ms/step - loss: 2.0734 - accuracy: 0.1500 - val_loss: 2.0798 - val_accuracy: 0.2000
Epoch 3/10
20/20 [==============================] - 3s 131ms/step - loss: 2.0634 - accuracy: 0.2500 - val_loss: 2.0939 - val_accuracy: 0.0667
Epoch 4/10
20/20 [==============================] - 1s 66ms/step - loss: 2.0863 - accuracy: 0.1125 - val_loss: 2.0991 - val_accuracy: 0.1333
Epoch 5/10
20/20 [==============================] - 2s 118ms/step - loss: 2.1006 - accuracy: 0.0000e+00 - val_loss: 2.0809 - val_accuracy: 0.0667
Epoch 6/10
20/20 [==============================] - 2s 90ms/step - loss: 2.0810 - accuracy: 0.0750 - val_loss: 2.0856 - val_accuracy: 0.1333
Epoch 7/10
20/20 [==============================] - 2s 83ms/step - loss: 2.0702 - accuracy: 0.1875 - val_loss: 2.0843 - val_accuracy: 0.0667
Epoc

**Tingkat Akurasi**


**Upload Image**

In [15]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline